In [1]:
%cd ../
%pwd

D:\Source\IndTB_ppa


'D:\\Source\\IndTB_ppa'

In [2]:
import json
from ppa import Cascade
import numpy as np
from scipy.integrate import solve_ivp

In [3]:
loc = 'India'

with open(f'out/cascade_2019/Cascade_{loc}.json', 'r') as f:
    cs = json.load(f)

cs = [Cascade.parse_obj(c) for c in cs]    

FileNotFoundError: [Errno 2] No such file or directory: 'out/cascade_2019/Cascade_India.json'

In [73]:
cas.revive()
cas.dict()

{'R_SelfCure': 0.2005383669927065,
 'R_Die_Asym': 0.04367120224719754,
 'R_Die_Sym': 0.16616892278636303,
 'R_Die_Tx': array([0.0952381 , 0.05633803, 0.16468999]),
 'R_Onset': 2.928247957517989,
 'R_Aware': 8.318529889638267,
 'R_CSI': 55.360119398158496,
 'R_ReCSI': 24.85807606603545,
 'P_Entry': array([0.48281849, 0.07557045, 0.44161106]),
 'P_Tr': array([[0.13654141, 0.12616838, 0.73729021],
        [0.62527285, 0.05475505, 0.3199721 ],
        [0.62527285, 0.05475505, 0.3199721 ]]),
 'P_Dx0': array([0.07843137, 0.07843137, 0.10209826]),
 'P_Dx1': array([[0.7481203 , 0.7481203 , 0.79872435],
        [0.7481203 , 0.7481203 , 0.79872435],
        [0.14068441, 0.14068441, 0.1794779 ]]),
 'P_TxI': array([0.94      , 0.95      , 0.74840515]),
 'R_Succ_Tx': array([2.        , 2.        , 5.84649478]),
 'R_LTFU_Tx': array([0.11904762, 0.14084507, 0.61758748]),
 'PrevUt': 0.0029064810040805504,
 'PrevTx': array([0.00053914, 0.00021619, 0.00036147]),
 'DetR': array([0.00127   , 0.0005    , 0

In [8]:
len([c for c in cs if c.R_ReCSI > 0])

300

In [9]:
Asym = 0
NotAware = 1
NotCS = 2
ExCS_pub = 3
ExCS_eng = 4
ExCS_pri = 5
ExCS = [ExCS_pub, ExCS_eng, ExCS_pri]
Tx_pub = 6
Tx_eng = 7
Tx_pri = 8
Tx = [Tx_pub, Tx_eng, Tx_pri]

N_States = 9

In [68]:
def dy(t, y, cas):
    dy = np.zeros_like(y)
    
    onset = cas.R_Onset * y[Asym] 
    aware = cas.R_Aware * y[NotAware]
    csi = cas.P_Entry * cas.R_CSI * y[NotCS]
    det0 = cas.P_Dx0 * csi
    fn0 = (1 - cas.P_Dx0) * csi

    recsi = cas.R_ReCSI * y[ExCS].reshape((-1, 1)) * cas.P_Tr
    det1 = cas.P_Dx1 * recsi
    fn1 = (1 - cas.P_Dx1) * recsi

    det = det0 + det1.sum(0)
    txi = det * cas.P_TxI
    
    dy[Asym] = det.sum() - onset
    dy[NotAware] = onset - aware
    dy[NotCS] = aware - csi.sum()
    dy[ExCS] = fn0 - recsi.sum(1) + fn1.sum(0)
    dy[Tx] = txi

    
    mu = np.zeros(N_States)
    mu[Asym] = cas.R_Die_Asym + cas.R_SelfCure
    mu[NotAware] = cas.R_Die_Sym + cas.R_SelfCure
    mu[NotCS] = cas.R_Die_Sym + cas.R_SelfCure
    mu[ExCS] = cas.R_Die_Sym + cas.R_SelfCure
    mu[Tx] = cas.R_Die_Tx + cas.R_Succ_Tx + cas.R_LTFU_Tx
    mu *= y
    
    dy -= mu
    dy[Asym] += mu[:Tx_pub].sum()

    return dy


def prop_det(y, cas):
    csi = cas.P_Entry * cas.R_CSI * y[NotCS]
    det0 = cas.P_Dx0 * csi
    fn0 = (1 - cas.P_Dx0) * csi

    recsi = cas.R_ReCSI * y[ExCS].reshape((-1, 1)) * cas.P_Tr
    det1 = cas.P_Dx1 * recsi
    fn1 = (1 - cas.P_Dx1) * recsi

    det = det0 + det1.sum(0)
    return det / det.sum()

In [69]:
y0 = np.zeros(N_States)
y0[Asym] = cas.PrevUt

ys = solve_ivp(dy, [0, 20], y0=y0, args = (cas, ), dense_output=True)

In [71]:
ys.y[:, -1]

prop_det(ys.y[:, -1], cas)

array([0.25597625, 0.10067381, 0.64334995])

In [72]:
cas.calc_prop_det()

array([0.25525724, 0.10081726, 0.6439255 ])